In [861]:
import nupack
from nupack import *
from collections import deque
import pandas as pd
import gspread
import RNA
from copy import copy, deepcopy
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
import numpy as np
import yaml

In [862]:
model1 = Model()
#Model parameters go here

salt_molarity = .01
temp = 25.0
md = RNA.md()
md.salt = salt_molarity
md.temperature = temp

#md_parameters = len(salt_molarity)
#end of parameters

sheet_link = 'https://docs.google.com/spreadsheets/d/1idf3GHur5Eneef_vnwpGD-dn_dhoEgqx/edit#gid=1170226371'
sheet_rows = 34 #how many rows we want to use
f_part = sheet_link.split('/edit#gid')[0]
csv_link = f_part + '/export?format=csv'
data = pd.read_csv(csv_link)
df = pd.read_csv(csv_link, usecols=['toehold_switch_sequence','target_sequence', 'location', 'switch_mfe_structure','target_mfe_structure'], nrows=sheet_rows)
# df.insert(len(df.columns), 'switch_energy', 1.0)
# df.insert(len(df.columns), 'target_energy', 1.0)
# df.insert(len(df.columns), 'switch_gc_count', -1)
# df.insert(len(df.columns), 'target_gc_count',-1)
# df.insert(len(df.columns), 'ensemble_defect_switch', 0)
# df.insert(len(df.columns), 'ensemble_defect_target', 0)
df.columns = df.columns.str.replace('location', 'target_location')

preliminary_data_names = ['switch_structure', 'target_structure', 'switch_energy', 'target_energy', 'switch_gc', 'target_gc', 'switch_ensemble_defect', 'target_ensemble_defect', 'switch_single_strandedness', 'target_single_strandedness']
old_data_names = ['salt_molarity', 'temperature (Celsius)', 'switch_row', 'target_row', 'target_location', 'toehold_switch_sequence', 'target_sequence', 'switch_mfe_structure', 'target_mfe_structure', 'switch_energy','switch_gc', 'switch_ensemble_defect', 'switch_single_strandedness', 'target_energy', 'target_gc', 'target_ensemble_defect', 'target_single_strandedness', 'valid_loop', 'loop_sequence', 'loop_structure', 'loop_energy', 'complex_nucelotides', 'complex_structure', 'complex_visualization_structure', 'complex_energy', 'complex_gc', 'complex_ensemble_defect', 'complex_single_strandedness']
finished_data_names = ['salt_molarity', 'temperature (Celsius)', 'switch_row', 'target_row', 'target_location', 'toehold_switch_sequence', 'target_sequence', 'complex_nucelotides', 'switch_structure', 'target_structure', 'complex_structure', 'complex_visualization_structure', 'switch_energy', 'target_energy', 'complex_energy', 'switch_gc', 'target_gc', 'complex_gc', 'switch_ensemble_defect', 'target_ensemble_defect', 'complex_ensemble_defect', 'switch_single_strandedness', 'target_single_strandedness', 'complex_single_strandedness', 'valid_loop', 'loop_sequence', 'loop_structure', 'loop_energy']

In [863]:
#computes ammounts of gc in sequence
def oneComputeGCtotal(seq):
    gc_content = 0
    total_bases = len(seq)
    idx = 0
    while idx < len(seq):
        seq_idx = seq[idx]
        if seq_idx == 'C' or seq_idx == 'G':
            gc_content += 1
        idx += 1
    gc_cont_perc = float(gc_content) / float(total_bases)
    return gc_cont_perc

In [864]:
def find_single_strandedness(base_pair_probability_matrix): #This was taken from Andrew's code, for some reason it's not working as intended... possible that diagonals means something different in Vienna
    assert base_pair_probability_matrix.ndim == 2, "matrix must be two dimensional"
    assert base_pair_probability_matrix.shape[0] == base_pair_probability_matrix.shape[1], "matrix must be square"

    sum_of_diagonal = np.sum(np.diag(base_pair_probability_matrix))
    length_of_sequence = base_pair_probability_matrix.shape[0]
    return sum_of_diagonal/length_of_sequence

In [865]:
def compute_preliminary_data(row): #This is the everything method for the panda applys
    switch_seq = row['toehold_switch_sequence']
    target_seq = row['target_sequence']
    switch_fc = RNA.fold_compound(switch_seq, md) #Fold compound takes a sequence and a model, we run methods on it to compute the things we want
    target_fc = RNA.fold_compound(target_seq, md)
    (switch_struct, switch_energy) = switch_fc.mfe() #mfe returns min free energy structure + energy
    (target_struct, target_energy) = target_fc.mfe()

    #Compute the gc count
    switch_gc = oneComputeGCtotal(switch_seq)
    target_gc = oneComputeGCtotal(target_seq)

    #we can use these for later, but we need to calculate BP probs and ensemble defect with pf first, which returns these results. Might as well store them for later
    #It basically computes the partiton function of a sequence (I don't exactly know what that means but it's needed)
    (switch_propensity_struct, switch_ensemble_energy) = switch_fc.pf()
    (target_propensity_struct, target_ensemble_energy) = target_fc.pf()

    #TODO: for later, figure out if we use MFE structure or ensemble structure for ensemble defect prediction
    #Computes ensemble defect (higher percent is bad)
    switch_ensemble_defect = switch_fc.ensemble_defect(switch_struct)
    target_ensemble_defect = target_fc.ensemble_defect(target_struct)

    #Below is to calculate base pair probability matrix and single strandedness OLD CODE
    switch_bpp = switch_fc.bpp() #Gives us base bair probability
    np_switch = np.asarray(switch_bpp)
    switch_single_strandedness = find_single_strandedness(np_switch)
    target_bpp = target_fc.bpp()
    np_target = np.asarray(target_bpp)
    target_single_strandedness = find_single_strandedness(np_target)
    
    #print(preliminary_data_names)
    return [switch_struct,target_struct, switch_energy, target_energy, switch_gc, target_gc, switch_ensemble_defect, target_ensemble_defect, switch_single_strandedness, target_single_strandedness]

In [866]:
df[preliminary_data_names] = df.apply(compute_preliminary_data, axis=1, result_type='expand')

In [867]:
df.head()

,target_location,toehold_switch_sequence,target_sequence,switch_mfe_structure,target_mfe_structure,switch_structure,target_structure,switch_energy,target_energy,switch_gc,target_gc,switch_ensemble_defect,target_ensemble_defect,switch_single_strandedness,target_single_strandedness
0,6946,GGGGAAGAGUCUAUUGAGCUAGUUGGUAGUCCUAUACAGAAACAGA...,AGGACTACCAACTAGCTCAATAGACTCTTC,..................(((((((((.((((((((.............,.((....))..(((......))).......,..................(((((((((.(((((((((............,..............................,-23.209999,0.00,0.464646,0.433333,0.075309,0.142913,0.0,0.0
1,6805,GGGGUUUCAUUUUAUUCCUAUACUGGCUGCUUAAUACAGAAACAGA...,TAAGCAGCCAGTATAGGAATAAAATGAAAC,(((((........)))))(((((((((((.((((((.............,.......((......)).............,(((((........)))))(((((((((((.(((((((............,.......((......)).............,-19.110001,-0.17,0.414141,0.333333,0.055719,0.062859,0.0,0.0
2,6804,GGGUUUCAUUUUAUUCCUAUACUGGCUGCUUAUAUACAGAAACAGA...,ATAAGCAGCCAGTATAGGAATAAAATGAAA,(((...........)))((((((((((..(((((((.............,........((......))............,.................((((((((((..((((((((............,........((......))............,-18.320000,-0.17,0.404040,0.300000,0.074547,0.062800,0.0,0.0
3,6771,GGGGACAAUUGAGAGAUUGUUACCAUAUAGCUAAUACAGAAACAGA...,TAGCTATATGGTAACAATCTCTCAATTGTC,........(((..((.(((((((((((((.((((((.............,.............(((((......))))).,.............((.(((((((((((((.(((((((............,.............(((((......))))).,-20.870001,-0.17,0.414141,0.333333,0.062054,0.202906,0.0,0.0
4,6712,GGGCUAUUAUAUAUCCCUUUGUUGGGUUUAUACAUACAGAAACAGA...,GTATAAACCCAACAAAGGGATATATAATAG,(((.(((((((((((((((((((.................))))))...,(((((..(((......))).))))).....,(((.(((((((((((((((((((..((........))...))))))...,.......(((......)))...........,-29.010000,-3.32,0.414141,0.300000,0.039752,0.164807,0.0,0.0


In [868]:
def compute_structure(seq):
    return RNA.fold_compound(seq,md).mfe()[0]

In [869]:
def compute_defect(sequence,structure):
    fc = RNA.fold_compound(sequence,md)
    return fc.ensemble_defect(structure)


In [870]:
# Really bad old code (then again my code is always bad)
#Set up of the table
# #Note: making seperate functions for each purpose as apply is optimized compared to for loops with pandas apparently. Reason why I have functions that compute similar things
# df['switch_mfe_structure'] = df['toehold_switch_sequence'].apply(compute_structure)
# df['target_mfe_structure'] = df['target_sequence'].apply(compute_structure)
# df['switch_gc_count'] = df['toehold_switch_sequence'].apply(oneComputeGCtotal)
# df['target_gc_count'] = df['target_sequence'].apply(oneComputeGCtotal)
# df['switch_energy'] = df['toehold_switch_sequence'].apply(computeEnergy)
# df['target_energy'] = df['target_sequence'].apply(computeEnergy)
# df['ensemble_defect_switch'] = df.apply(lambda x: compute_defect(x['toehold_switch_sequence'], x['switch_mfe_structure']), axis=1)
# df.head()

In [871]:
def test_valid_notation(d): #just sees that every opening parenthesis has a closing parethesis (i.e is valid notation)
    q = deque()
    for let in d:
        if let == '(':
            q.append(let)
        elif let == ')':
            if (len(q)==0):
                return 1 #if extra closed parethesis
            q.pop()
    if(len(q)==0):
        return 0 #if valid
    return 2 #if extra open parethesis

In [872]:
def stem_loop_eval(s): #Stem loop region is 35-48 nucleotide inclusive
    start = 35
    end = 49
    strand_sub = s[start:end]
    ind = 1
    limit = 3 #Limit is 3 because seems like a good num i suppose
    result = test_valid_notation(strand_sub)
    #print(strand_sub)
    while result!=0 and ind<=limit: #there might be off by 1 errors, so we are gonna add up to the limit that many nueclotides to the sequence to try to get a valid sequence
        if result==1:
            start-=1
        else:
            end+=1
        strand_sub = s[start:end]
        #print(strand_sub) 
        result = test_valid_notation(strand_sub)
        #print(ind)
        ind+=1
    if result == 0:
        return True, start, end
    return False, -1, -1
        

In [873]:
#These are the col names of the table being filled in, later the order of the columns will be changed according to the 'finished data names up top
col_names = ['switch_row','target_row', 'target_location', 'toehold_switch_sequence', 'target_sequence'] + preliminary_data_names[0:4] + ['valid_loop', 'loop_sequence', 'loop_structure','loop_energy','complex_nucelotides','complex_structure', 'complex_visualization_structure','complex_energy','complex_gc'] + preliminary_data_names[4:8] + ['complex_ensemble_defect'] + preliminary_data_names[8:] + ['complex_single_strandedness']
print(col_names)
len(col_names)

['switch_row', 'target_row', 'target_location', 'toehold_switch_sequence', 'target_sequence', 'switch_structure', 'target_structure', 'switch_energy', 'target_energy', 'valid_loop', 'loop_sequence', 'loop_structure', 'loop_energy', 'complex_nucelotides', 'complex_structure', 'complex_visualization_structure', 'complex_energy', 'complex_gc', 'switch_gc', 'target_gc', 'switch_ensemble_defect', 'target_ensemble_defect', 'complex_ensemble_defect', 'switch_single_strandedness', 'target_single_strandedness', 'complex_single_strandedness']


26

In [874]:
def fill_table(arr_loc, arr, data):
    for k in range(len(col_names)):
        # print(len(arr[arr_loc]))
        #print(len(data))
        arr[arr_loc][k] = data[k]  
    

In [875]:
#TODO: Append data to a list and then assign it
#ALSO look at why tf switch gc and target was changed
ribo_size = len(df.dropna().toehold_switch_sequence)
switch_size = len(df.dropna().target_sequence)
vArr = [[0 for i in range (len(col_names))] for j in range (ribo_size*switch_size)] #fix later lol
test = deepcopy(vArr)

#0 is switch row
#1 is target row
#2 is target location
#3 is toehold switch sequence
#4 is target sequence
#5 is switch mfe struct
#6 is target mfe struct
#7 is switch energy
#8 is target energy
#9 is valid stem loop
#10 is stem loop sequence
#11 is stem loop structure
#12 is stem loop energy
#13 is complex nucelotides
#14 is complex structure
#15 is complex visualization
#16 is complex energy
#17 is complex gc count
#18 is switch gc count
#19 is target gc count
#20 is switch ensemble defect
#21 is target ensemble defect
#22 is complex ensemble defect
#23 is switch single strandness
#24 is target single strandness
#25 is complex single strandness

for x in range(ribo_size):
    switch_struct = df.at[x,'switch_mfe_structure']
    switch_seq = df.at[x,'toehold_switch_sequence']

    for y in range(switch_size):
        data_list = []
        target_seq = df.at[y,'target_sequence']
        target_struct = df.at[y,'target_mfe_structure']
        arr_loc = x * switch_size + y

        vArr[arr_loc][0] = x
        vArr[arr_loc][1] = y
        vArr[arr_loc][2] = df.at[y,'target_location']
        vArr[arr_loc][3] = switch_seq
        vArr[arr_loc][4] = target_seq
        vArr[arr_loc][5] = switch_struct
        vArr[arr_loc][6] = target_struct
        vArr[arr_loc][7] = df.at[x,'switch_energy']
        vArr[arr_loc][8] = df.at[y,'target_energy']

        data_list.extend([x, y, df.at[y, 'target_location'], switch_seq, target_seq, switch_struct, target_struct, df.at[x,'switch_energy'], df.at[y,'target_energy']])

        #This is scuffed, there should be a better way but this will do for now
        #This evaluates the stem loop and its free energy, if it's valid
        outcome = stem_loop_eval(switch_struct)
        vArr[arr_loc][9] = validity = outcome[0]
        data_list.append(outcome[0])
        if(validity==False): #if not valid from stem loop test
            vArr[arr_loc][10] = vArr[arr_loc][11] = vArr[arr_loc][12] = 'N/A' 
            data_list.extend(['N/A', 'N/A', 'N/A'])
        else:
            vArr[arr_loc][10] = sub_seq = switch_seq[outcome[1]:outcome[2]]
            vArr[arr_loc][11] = sub_struct = switch_struct[outcome[1]:outcome[2]]
            vArr[arr_loc][12] = RNA.fold_compound(sub_seq,md).mfe()[1]
            data_list.extend([switch_seq[outcome[1]:outcome[2]], switch_struct[outcome[1]:outcome[2]], RNA.fold_compound(sub_seq,md).mfe()[1]])
        
        vArr[arr_loc][13] = target_seq+'+'+switch_seq
        data_list.append(target_seq+'+'+switch_seq)

        # NOTE SOME OF THE SEQUENCES HAVE T's IN THEM INSTEAD OF U's... WE GOTTA REPLACE THEM, Orignial is still listed in table
        new_switch_seq = switch_seq.replace('T','U')

        fc = RNA.fold_compound(target_seq+'&'+new_switch_seq, md)
        (v_complex_struct, v_complex_mfe) = fc.mfe()
        vArr[arr_loc][14] = v_complex_struct
        vArr[arr_loc][15] = v_complex_struct.replace('+','&')
        vArr[arr_loc][16] = v_complex_mfe
        vArr[arr_loc][17] = oneComputeGCtotal(target_seq+switch_seq)
        vArr[arr_loc][18] = df.at[x,'switch_gc']
        vArr[arr_loc][19] = df.at[y, 'target_gc']

        data_list.extend([v_complex_struct, v_complex_struct.replace('+','&'), v_complex_mfe, oneComputeGCtotal(target_seq+switch_seq),df.at[x,'switch_gc'] ,df.at[y, 'target_gc']])
        
        #ensemble defect/parititon function calculations
        vArr[arr_loc][20] = df.at[x, 'switch_ensemble_defect']
        vArr[arr_loc][21] = df.at[y, 'target_ensemble_defect']
        (complex_propensity, complex_ensemble_energy) = fc.pf()
        vArr[arr_loc][22] = fc.ensemble_defect(v_complex_struct)
        data_list.extend([df.at[x, 'switch_ensemble_defect'], df.at[y, 'target_ensemble_defect'], fc.ensemble_defect(v_complex_struct)])

        #Single Strandedness old stuff
        vArr[arr_loc][23] = df.at[x, 'switch_single_strandedness']
        vArr[arr_loc][24] = df.at[y, 'target_single_strandedness']
        complex_bpp = fc.bpp()
        np_complex = np.asarray(complex_bpp)
        vArr[arr_loc][25] = find_single_strandedness(np_complex)
        data_list.extend([df.at[x, 'switch_single_strandedness'],df.at[y, 'target_single_strandedness'], find_single_strandedness(np_complex)])

        fill_table(arr_loc, test, data_list)
        

        

vArr.insert(0, col_names)
test.insert(0, col_names)
#print(vArr)
results = pd.DataFrame(vArr)
testing = pd.DataFrame(test)

In [876]:
# legacy code, use later for reference i suppose
# ribo_size = len(df.dropna().toehold_switch_sequence)
# switch_size = len(df.dropna().target_sequence)
# arr = [[0 for i in range (len(col_names))] for j in range (ribo_size*switch_size)]
# vArr = deepcopy(arr)

# #0 is switch row
# #1 is target row
# #2 is target location
# #3 is toehold switch sequence
# #4 is target sequence
# #5 is switch mfe struct
# #6 is target mfe struct
# #7 is switch energy
# #8 is target energy
# #9 is valid stem loop
# #10 is stem loop sequence
# #11 is stem loop structure
# #12 is stem loop energy
# #13 is complex nucelotides
# #14 is complex structure
# #15 is complex visualization
# #16 is complex energy

# for x in range(ribo_size):
#     switch_struct = df.at[x,'switch_mfe_structure']
#     switch_seq = df.at[x,'toehold_switch_sequence']
#     if(df.at[x,'switch_energy']>0): #if switch energy not computed... surely there is a better way of doing this...
#         #Old code for switch energy df.at[x,'switch_energy'] = structure_energy(strands=switch_seq, structure = switch_struct, model = model1)
#         df.at[x,'switch_energy'] = RNA.fold_compound(switch_seq,md).mfe()[1]
#     for y in range(switch_size):
#         target_seq = df.at[y,'target_sequence']
#         target_struct = df.at[y,'target_mfe_structure']
#         if(df.at[y,'target_energy']>0): #if target energy not already computed
#             #Old code for target energy df.at[y,'target_energy'] = structure_energy(strands=target_seq, structure = target_struct, model = model1)
#             df.at[y,'target_energy'] = RNA.fold_compound(target_seq, md).mfe()[1]
#         arr_loc = x * switch_size + y
#         arr[arr_loc][0] = vArr[arr_loc][0] = x
#         arr[arr_loc][1] = vArr[arr_loc][1] = y
#         arr[arr_loc][2] = vArr[arr_loc][2] = df.at[y,'target_location']
#         arr[arr_loc][3] = vArr[arr_loc][3] = switch_seq
#         arr[arr_loc][4] = vArr[arr_loc][4] = target_seq
#         arr[arr_loc][5] = vArr[arr_loc][5] = switch_struct
#         arr[arr_loc][6] = vArr[arr_loc][6] = target_struct
#         arr[arr_loc][7] = vArr[arr_loc][7] = df.at[x,'switch_energy']
#         arr[arr_loc][8] = vArr[arr_loc][8] = df.at[y,'target_energy']

#         outcome = stem_loop_eval(switch_struct)
#         arr[arr_loc][9]= vArr[arr_loc][9] = validity = outcome[0]
#         if(validity==False): #if not valid from stem loop test
#             arr[arr_loc][10] = arr[arr_loc][11] = arr[arr_loc][12] = vArr[arr_loc][10] = vArr[arr_loc][11] = vArr[arr_loc][12] = 'N/A'
#         else:
#             arr[arr_loc][10] = vArr[arr_loc][10] = sub_seq = switch_seq[outcome[1]:outcome[2]]
#             arr[arr_loc][11] = vArr[arr_loc][11] = sub_struct = switch_struct[outcome[1]:outcome[2]]
#             arr[arr_loc][12] = structure_energy(strands=[sub_seq], structure = sub_struct, model = model1)
#             vArr[arr_loc][12] = RNA.fold_compound(sub_seq,md).mfe()[1]
        
#         vArr[arr_loc][13] = arr[arr_loc][13] = target_seq+switch_seq

#         #s stands for switch strand
#         #t stands for target strand
#         # NOTE TO SELF HERE: SOME OF THE SEQUENCES HAVE T's IN THEM INSTEAD OF U's... WE GOTTA REPLACE THEM, Orignial is still listed in table
#         new_switch_seq = switch_seq.replace('T','U')
#         s = Strand(new_switch_seq, name = 's')
#         t = Strand(target_seq, name = 't')
#         c1 = Complex([t,s], name = 't+s')
#         #below is test, remove later
#         fc = RNA.fold_compound(target_seq+'&'+new_switch_seq, md)
#         (v_complex_struct, v_complex_mfe) = fc.mfe()
#         vArr[arr_loc][14] = v_complex_struct
#         vArr[arr_loc][15] = v_complex_struct.replace('+','&')
#         vArr[arr_loc][16] = v_complex_mfe
#         #End of test
#         combine_set = ComplexSet(strands=[s,t], complexes = SetSpec(include=[c1]))
#         # for ab in combine_set:
#         #     print(ab)
#         # print('_______')
#         set_result = complex_analysis(complexes=combine_set, model = model1, compute = ['mfe'])
#         # for z in set_result:
#         #     print(z)
#         # print('------')
#         complex_struct = set_result['(t+s)'].mfe[0].structure.dotparensplus()
#         complex_mfe = set_result['(t+s)'].mfe[0].energy

#         arr[arr_loc][14] = complex_struct
#         arr[arr_loc][15] = complex_struct.replace('+','&')
#         arr[arr_loc][16] = complex_mfe

# arr.insert(0, col_names)
# vArr.insert(0, col_names)
# print(arr)
# print(vArr)
# test = pd.DataFrame(arr)
# results = pd.DataFrame(vArr)


In [877]:

results = results.rename(columns=results.iloc[0])
results = results.drop(0)
results = results.reset_index(drop = True)
results.insert(0, 'salt_molarity', salt_molarity)
results.insert(1, 'temperature (Celsius)', temp)
#display(results)

In [878]:
testing = testing.rename(columns=testing.iloc[0])
testing = testing.drop(0)
testing = testing.reset_index(drop = True)
testing.insert(0, 'salt_molarity', salt_molarity)
testing.insert(1, 'temperature (Celsius)', temp)

In [879]:
original_cols = results.columns.to_list()
print(original_cols)



['salt_molarity', 'temperature (Celsius)', 'switch_row', 'target_row', 'target_location', 'toehold_switch_sequence', 'target_sequence', 'switch_structure', 'target_structure', 'switch_energy', 'target_energy', 'valid_loop', 'loop_sequence', 'loop_structure', 'loop_energy', 'complex_nucelotides', 'complex_structure', 'complex_visualization_structure', 'complex_energy', 'complex_gc', 'switch_gc', 'target_gc', 'switch_ensemble_defect', 'target_ensemble_defect', 'complex_ensemble_defect', 'switch_single_strandedness', 'target_single_strandedness', 'complex_single_strandedness']


In [880]:
results = results[finished_data_names]
testing = testing[finished_data_names]

In [881]:
# display(results)


In [882]:
# display(testing)

In [883]:
results_sorted = results.sort_values(by = ['complex_energy'])
#display(results_sorted)

In [884]:
# html = test.to_html()

# text_file = open('index.html','w')
# text_file.write(html)
# text_file.close()

In [885]:
#test.to_csv('nupack_data.csv')
results.to_csv('vienna_data.csv')
results.to_csv('tab_sep_vienna.csv', sep='\t')
stream = open('YAML_vienna.yaml', 'w')
yaml.dump(results.to_dict(orient='records'), stream, sort_keys=False)
yaml_data = yaml.dump(results.to_dict(orient='records'), sort_keys=False)
#print(yaml_data)

#test2.to_csv('test_data.csv')